In [8]:
import numpy as np
from qutip import basis, ket2dm, tensor, qeye, fidelity
import itertools


H = basis(2,0)
V = basis(2,1)

PAIR_RATE   = 15000   # entangled pairs / second
DARK_RATE   = 1000    # dark counts / second
EFFICIENCY  = 0.10    # 10%
DEAD_TIME   = 4e-6    # 4 microseconds
T_RUN       = 30      # 30 seconds
COINC_WIN   = 1e-9    # 1 ns

# For a CHSH angles:
# Alice: a=0°, a'=45° => [0, pi/4]
# Bob:   b=22.5°, b'=67.5° => [pi/8, 3*pi/8]
ALICE_ANGLES = [0, np.pi/4]
BOB_ANGLES   = [np.pi/8, 3*np.pi/8]



### Define the States

In [9]:
def build_states():
    # |Phi+>
    phi_plus_ket = (tensor(H, H) + tensor(V, V)).unit()
    phi_plus = ket2dm(phi_plus_ket)

    # rho = 0.7 |Phi+><Phi+| + 0.15 |HV><HV| + 0.15 |VH><VH|
    hv = ket2dm(tensor(H,V))
    vh = ket2dm(tensor(V,H))
    rho = 0.7*phi_plus + 0.15*hv + 0.15*vh

    # |phi> = (1/sqrt(5))|HH> + (2/sqrt(5))|VV>
    phi_ket = (1/np.sqrt(5)*tensor(H,H) + 2/np.sqrt(5)*tensor(V,V)).unit()
    phi_dm  = ket2dm(phi_ket)

    # rho' = 0.7|phi><phi| + 0.15|HV><HV| + 0.15|VH><VH|
    rho_prime = 0.7*phi_dm + 0.15*hv + 0.15*vh

    return {
        "Phi+": phi_plus,
        "rho": rho,
        "phi": phi_dm,
        "rho_prime": rho_prime
    }

global_phi_plus_ket = (tensor(H,H) + tensor(V,V)).unit()
global_phi_plus_dm  = ket2dm(global_phi_plus_ket)

### Measurements

In [10]:
def projector(theta):
    st = np.cos(theta)*H + np.sin(theta)*V
    return ket2dm(st.unit())

def measure_2qubit(rho_2q, thetaA, thetaB):
    """
    Perform a 2-qubit measurement with measurement angles (thetaA, thetaB).
    """
    A_plus = projector(thetaA)
    B_plus = projector(thetaB)

    A_minus = qeye(2) - A_plus
    B_minus = qeye(2) - B_plus

    # 2-qubit projectors
    P_pp = tensor(A_plus,  B_plus)   # ++
    P_pm = tensor(A_plus,  B_minus)  # +-
    P_mp = tensor(A_minus, B_plus)   # -+
    P_mm = tensor(A_minus, B_minus)  # --

    p_pp = (P_pp * rho_2q).tr().real
    p_pm = (P_pm * rho_2q).tr().real
    p_mp = (P_mp * rho_2q).tr().real
    p_mm = (P_mm * rho_2q).tr().real

    probs = [p_pp, p_pm, p_mp, p_mm]
    outcomes = ['++','+-','-+','--']
    return np.random.choice(outcomes, p=probs)

### Entangled Pair and Dark Count Generation

In [11]:
def generate_pairs():
    n_pairs = np.random.poisson(PAIR_RATE * T_RUN)
    times = np.random.rand(n_pairs)*T_RUN
    times.sort()
    pairs = []
    for t in times:
        a = np.random.choice(ALICE_ANGLES)
        b = np.random.choice(BOB_ANGLES)
        pairs.append({'time': t, 'thetaA': a, 'thetaB': b})
    return pairs

def generate_dark_events(is_alice):
    evts = []
    n_dark = np.random.poisson(DARK_RATE * T_RUN)
    times = np.random.rand(n_dark)*T_RUN
    times.sort()
    last_d = -np.inf
    for t in times:
        if t - last_d >= DEAD_TIME and np.random.rand()<EFFICIENCY:
            # random angle
            angle = np.random.choice(ALICE_ANGLES if is_alice else BOB_ANGLES)
            sgn = '+' if np.random.rand()<0.5 else '-'
            evts.append({
                'time': t,
                'theta': angle,
                'outcome': sgn,
                'dark': True,
                'pair_id': -1
            })
            last_d = t
    return evts

def correlation(cd):
    s = sum(cd.values())
    if s==0: return 0
    return (cd['++']+cd['--'] - cd['+-']-cd['-+'])/s

### Simulation 

In [12]:

def simulate_bell_test(rho_2q):
    pairs = generate_pairs()

    # Store detection events for Alice, Bob
    alice_events = []
    bob_events   = []
    last_a = -np.inf
    last_b = -np.inf

    for i, p in enumerate(pairs):
        t = p['time']
        # check if Alice detects
        detectA = False
        if t - last_a >= DEAD_TIME and np.random.rand() < EFFICIENCY:
            detectA = True
        # check if Bob detects
        detectB = False
        if t - last_b >= DEAD_TIME and np.random.rand() < EFFICIENCY:
            detectB = True

        if detectA and detectB:
            outcome_2q = measure_2qubit(rho_2q, p['thetaA'], p['thetaB'])
            last_a = t
            last_b = t

            alice_events.append({
                'time': t,
                'theta': p['thetaA'],
                'outcome': outcome_2q[0],
                'dark': False,
                'pair_id': i
            })
            bob_events.append({
                'time': t,
                'theta': p['thetaB'],
                'outcome': outcome_2q[1],
                'dark': False,
                'pair_id': i
            })

        else:
            # if only Alice detects
            if detectA:
                last_a = t
                sgn = '+' if np.random.rand()<0.5 else '-'
                alice_events.append({
                    'time': t,
                    'theta': p['thetaA'],
                    'outcome': sgn,
                    'dark': False,
                    'pair_id': i
                })
            # if only Bob detects
            if detectB:
                last_b = t
                sgn = '+' if np.random.rand()<0.5 else '-'
                bob_events.append({
                    'time': t,
                    'theta': p['thetaB'],
                    'outcome': sgn,
                    'dark': False,
                    'pair_id': i
                })
            # if neither detects => no events

    # generate dark counts
    alice_dark = generate_dark_events(True)
    bob_dark   = generate_dark_events(False)

    # Merge
    alice_all = alice_events + alice_dark
    bob_all   = bob_events   + bob_dark
    alice_all.sort(key=lambda e: e['time'])
    bob_all.sort(key=lambda e: e['time'])

    # time-based coincidence
    i, j = 0, 0
    coinc_map = {}
    for a_ in ALICE_ANGLES:
        for b_ in BOB_ANGLES:
            coinc_map[(a_, b_)] = {'++':0, '+-':0, '-+':0, '--':0}

    while i < len(alice_all) and j < len(bob_all):
        da = alice_all[i]
        db = bob_all[j]
        dt = da['time'] - db['time']
        if abs(dt) <= COINC_WIN:
            # we have a coincidence
            pair_key = (da['theta'], db['theta'])
            if pair_key in coinc_map:
                combo = da['outcome'] + db['outcome'] 
                if combo in coinc_map[pair_key]:
                    coinc_map[pair_key][combo]+=1
            i+=1
            j+=1
        elif dt<0:
            i+=1
        else:
            j+=1


    E_ab   = correlation(coinc_map[(ALICE_ANGLES[0], BOB_ANGLES[0])])
    E_abp  = correlation(coinc_map[(ALICE_ANGLES[0], BOB_ANGLES[1])])
    E_apb  = correlation(coinc_map[(ALICE_ANGLES[1], BOB_ANGLES[0])])
    E_apbp = correlation(coinc_map[(ALICE_ANGLES[1], BOB_ANGLES[1])])

    S = E_ab - E_abp + E_apb + E_apbp

    # Detection rates
    alice_rate = len(alice_all)/T_RUN
    bob_rate   = len(bob_all)/T_RUN
    total_coinc = 0
    for cdict in coinc_map.values():
        total_coinc += sum(cdict.values())
    coinc_rate = total_coinc / T_RUN

    fid_qutip = fidelity(rho_2q, global_phi_plus_dm)

    return {
        'alice_rate': alice_rate,
        'bob_rate':   bob_rate,
        'coinc_rate': coinc_rate,
        'S': S,
        'fidelity': fid_qutip
    }

In [13]:
np.random.seed(42) 
states = build_states()
results = {}

for name, rho_2q in states.items():
    res = simulate_bell_test(rho_2q)
    results[name] = res

print("=== Final Results ===")
for name, r in results.items():
    print(f"  State: {name}")
    print(f"  Alice Rate (counts/s): {r['alice_rate']:.2f}")
    print(f"  Bob Rate   (counts/s): {r['bob_rate']:.2f}")
    print(f"  Coinc Rate (counts/s): {r['coinc_rate']:.2f}")
    print(f"  Fidelity:       {r['fidelity']:.3f}")
    print(f"  CHSH S:                {r['S']:.3f}\n")


=== Final Results ===
  State: Phi+
  Alice Rate (counts/s): 1596.37
  Bob Rate   (counts/s): 1585.97
  Coinc Rate (counts/s): 151.37
  Fidelity:       1.000
  CHSH S:                2.795

  State: rho
  Alice Rate (counts/s): 1582.63
  Bob Rate   (counts/s): 1592.27
  Coinc Rate (counts/s): 146.80
  Fidelity:       0.837
  CHSH S:                1.493

  State: phi
  Alice Rate (counts/s): 1587.77
  Bob Rate   (counts/s): 1603.57
  Coinc Rate (counts/s): 148.63
  Fidelity:       0.949
  CHSH S:                2.519

  State: rho_prime
  Alice Rate (counts/s): 1591.10
  Bob Rate   (counts/s): 1585.27
  Coinc Rate (counts/s): 146.80
  Fidelity:       0.794
  CHSH S:                1.325



### Find state with mimimum S value

In [14]:
def phi_plus():
    return (tensor(H,H)+tensor(V,V)).unit()
phiP = ket2dm(phi_plus())

def werner_state(p):
    id_2q = qeye([2,2])/4
    return p*phiP + (1-p)*id_2q

pvals = np.linspace(0.70,0.72,50)
min_S = float('inf')
min_p = None

for p in pvals:
    stateW = werner_state(p)
    rW = simulate_bell_test(stateW)
    current_S = rW['S']
    print(f" p={p:.8f}, S={current_S:.8f}")

    if current_S > 2.0 and current_S < min_S:
        min_S = current_S
        min_p = p

if min_p is not None:
    print(f"\nMinimum violating S: {min_S:.3f} at p = {min_p:.10f}")
else:
    print("No Bell inequality violation detected in the tested range.")

 p=0.70000000, S=1.96104916
 p=0.70040816, S=1.93423523
 p=0.70081633, S=1.90219861
 p=0.70122449, S=1.95381967
 p=0.70163265, S=1.89942025
 p=0.70204082, S=2.07407047
 p=0.70244898, S=2.01793787
 p=0.70285714, S=1.99925098
 p=0.70326531, S=2.09841644
 p=0.70367347, S=1.97545409
 p=0.70408163, S=2.03188354
 p=0.70448980, S=1.95980202
 p=0.70489796, S=1.93204160
 p=0.70530612, S=1.92942723
 p=0.70571429, S=1.91006761
 p=0.70612245, S=2.04680497
 p=0.70653061, S=1.94121123
 p=0.70693878, S=1.97881655
 p=0.70734694, S=1.90698966
 p=0.70775510, S=2.00066179
 p=0.70816327, S=2.01153908
 p=0.70857143, S=2.01596105
 p=0.70897959, S=1.96336837
 p=0.70938776, S=2.02804590
 p=0.70979592, S=2.03342522
 p=0.71020408, S=2.01399368
 p=0.71061224, S=2.03525096
 p=0.71102041, S=2.05766540
 p=0.71142857, S=2.06962530
 p=0.71183673, S=1.91627160
 p=0.71224490, S=2.05007653
 p=0.71265306, S=1.93421451
 p=0.71306122, S=1.92867676
 p=0.71346939, S=1.99947883
 p=0.71387755, S=2.01690349
 p=0.71428571, S=2.0

Bad pipe message: %s [b'\x91\x9fJ\x94\xcf}\xa4\xfb\xf7\x17>S\x14!\xb1\xee\xe5) F-\xc2fc\xd07\xa3\xd9\x0f\xe5\x0f\xb5\\>s\x17\xf3\xe3\x07\xe3\xf5\xa4C\x83\xdf\xa1\xc7\xdc\xb0)\xf2\x00 \xca\xca\x13\x01\x13\x02\x13\x03\xc0+\xc0/\xc0,\xc00\xcc\xa9\xcc\xa8\xc0\x13\xc0\x14\x00\x9c\x00\x9d\x00/\x005\x01\x00\x06s\x9a\x9a\x00\x00\x00\x1b\x00\x03\x02\x00\x02\x003\x04\xef\x04\xed::\x00\x01\x00\x11\xec\x04\xc0\x93\xf9\xc6\xf7u\x80p\xb9m\x84\xdaP\xbfF\xbe\xcf\x9b\xb7\x08e\x01h\xea\xa4s\x00\\\xa2\x9c\xbe\x94\xdb\xbf_\x14\t\xde']
Bad pipe message: %s [b'\xa9\xab\x8a\xdf]\xe1\xb9i4\x00\xfe\xbc\xb3\xcf', b'\xc9\xf2 b\xd3\x95\xa5G\xb3N;\xd2\x1e\xdd\x98\xde,9\tD\x7f\x9d\x18w\x91\xf0\xd3\x9fc\xdf\xd0C\xe62x\x00 \xca\xca\x13\x01\x13\x02']
Bad pipe message: %s [b'[\xb4,\xdcC\xca?c\xa0\xfa \x9ap\xe4\x1dnv\x00Ft[\xbeqf\nX\xafUjm\x97zn\xc5\xe0\xb4\x98\x05>\x97\xfc\x8e}\xe64\xde\x9b\x98\x8cF\x8a7\nA\xf5I\x80G"\x04\xa1\xa6\xc7\xcd\x9c@C\xfa\x1a*\xd0K\x17\xf8\x98\x84\xc4e\xf8\xd1\x00\xe8y\x85\\9#,B\x8az$\x8d\xb9\